# EOLES_elecRES in Julia

[EOLES_elecRES](https://github.com/BehrangShirizadeh/EOLES_elecRES) ported to Julia, with the [JuMP]() package.

Precisely: EOLES_elecRES.gms

PH, Feb 2021

In [51]:
using CSV, DataFrames

## Model data

### Sets

#### hours and months

sets `h /0*8759/`

         first(h)        'first hour'
         last(h)         'last hour'
         
         first(h) = ord(h)=1;
last(h) = ord(h)=card(h);

alias(h,hh);

In [43]:
h = 0:8759

0:8759

In [45]:
m = [:jan, :feb, :mar, :apr, :may, :jun, :jul, :aug, :sep, :oct, :nov, :dec]

12-element Array{Symbol,1}:
 :jan
 :feb
 :mar
 :apr
 :may
 :jun
 :jul
 :aug
 :sep
 :oct
 :nov
 :dec

#### technology sets

All tech

In [ ]:
tec = [:offshore, :onshore, :pv, :river, :lake, :biogas, :gas, :phs, :battery, :methanation] 

Subsets of tec

In [49]:
# power plants
gen = [:offshore, :onshore, :pv, :river, :lake, :biogas, :gas]

7-element Array{Symbol,1}:
 :offshore
 :onshore
 :pv
 :river
 :lake
 :biogas
 :gas

### Parameters

parameter month(h)  /0*743 1, 744*1439 2, 1440*2183 3, 2184*2903 4
                    2904*3647 5, 3648*4367 6, 4368*5111 7, 5112*5855 8
                    5856*6575 9, 6576*7319 10, 7320*8039 11, 8040*8759 12/

parameter `load_factor(vre,h)` 'Production profiles of VRE'

from file `inputs/vre_profiles2006.csv`

In [14]:
load_factor = DataFrame(CSV.File("inputs/vre_profiles2006.csv", header=["vre", "hour", "cf"]))
load_factor[1:5,:]

,vre,hour,cf
,String,Int64,Float64
1,offshore,0,0.865143
2,offshore,1,0.88
3,offshore,2,0.88
4,offshore,3,0.873429
5,offshore,4,0.852


In [15]:
load_factor_wide = unstack(load_factor, :hour, :vre, :cf)
load_factor_wide[1:5,:]

,hour,offshore,onshore,pv
,Int64,Float64?,Float64?,Float64?
1,0,0.865143,0.511702,0.0
2,1,0.88,0.52791,0.0
3,2,0.88,0.537973,0.0
4,3,0.873429,0.541571,0.0
5,4,0.852,0.538737,0.0


In [22]:
size(load_factor), size(load_factor_wide)

((26280, 3), (8760, 4))

Notice that the data type is `Union{Missing, Float64}` rather than `Float64`. Are there missing bits? Answer: no. Counting gives 0, and conversion to Float64 does not fail.

In [16]:
load_factor_wide.offshore[1:5]

5-element Array{Union{Missing, Float64},1}:
 0.865142857
 0.88
 0.88
 0.873428571
 0.852

In [25]:
length(findall(x -> ismissing(x), load_factor_wide.pv))

0

In [40]:
load_factor = convert(Matrix{Float64}, load_factor_wide[:,2:end])'
load_factor

3×8760 LinearAlgebra.Adjoint{Float64,Array{Float64,2}}:
 0.865143  0.88     0.88      0.873429  …  0.990286  0.991     0.988143
 0.511702  0.52791  0.537973  0.541571     0.826158  0.825222  0.818481
 0.0       0.0      0.0       0.0          0.0       0.0       0.0

In [ ]:
Parameter lake_inflows(m) 'monthly lake inflows in GWh'
/
$ondelim
$include  inputs/lake_inflows.csv
$offdelim
/ ;

In [ ]:
parameter gene_river(h) 'hourly run of river power generation in GWh'
/
$ondelim
$include  inputs/run_of_river.csv
$offdelim
/ ;

In [ ]:
parameter epsilon(vre) 'additional FRR requirement for variable renewable energies because of forecast errors'
/
$ondelim
$include  inputs/reserve_requirements.csv
$offdelim
/ ;

In [ ]:
parameter capa_ex(tec) 'existing capacities of the technologies by December 2017 in GW'
/
$ondelim
$include  inputs/existing_capas.csv
$offdelim
/ ;

In [ ]:
parameter capa_max(tec) 'maximum capacities of the technologies in GW'
/
$ondelim
$include  inputs/max_capas.csv
$offdelim
/ ;

In [ ]:
parameter capex(tec) 'annualized power capex cost in M€/GW/year'
/
$ondelim
$include  inputs/annuities.csv
$offdelim
/ ;

In [ ]:
parameter capex_en(str) 'annualized energy capex cost of storage technologies in M€/GWh/year'
/
$ondelim
$include  inputs/str_annuities.csv
$offdelim
/ ;

In [ ]:
parameter fOM(tec) 'annualized fixed operation and maintenance costs M€/GW/year'
/
$ondelim
$include  inputs/fO&M.csv
$offdelim
/ ;

In [ ]:
Parameter vOM(tec) 'Variable operation and maintenance costs in M€/GWh'
/
$ondelim
$include  inputs/vO&M.csv
$offdelim
/ ;

```gams
parameter s_capex(str) 'charging related annuity of storage in M€/GW/year' /PHS 0, battery 0, methanation 84.16086/;
parameter s_opex(str)    'charging related fOM of storage in M€/GW/year'   /PHS 0, battery 0, methanation 59.25/;
parameter eta_in(str) 'charging efifciency of storage technologies' /PHS 0.95, battery 0.9, methanation 0.59/;
parameter eta_out(str) 'discharging efficiency of storage technolgoies' /PHS 0.9, battery 0.95, methanation 0.45/;
scalar pump_capa 'pumping capacity in GW' /9.3/;
scalar max_phs 'maximum volume of energy can be stored in PHS reservoir in TWh' /0.18/;
scalar max_biogas 'maxium energy can be generated by biogas in TWh' /15/;
scalar load_uncertainty 'uncertainty coefficient for hourly demand' /0.01/;
scalar delta 'load variation factor'     /0.1/;
parameter fixed_costs(tec) 'yearly fixed cost of each tec in M€/GW/year' ;
fixed_costs(tec) = capex(tec) + fOM(tec);
```

## Model

### Variables

```
variables        GENE(tec,h)     'hourly energy generation in TWh'
                 CAPA(tec)       'overal yearly installed capacity in GW'
                 STORAGE(str,h)  'hourly electricity input of battery storage GW'
                 S(str)          'charging power capacity of each storage technology'
                 STORED(str,h)   'energy stored in each storage technology in GWh'
                 CAPACITY(str)   'energy volume of storage technologies in GWh'
                 RSV(frr,h)      'required upward frequency restoration reserve in GW'
                 COST            'final investment cost in b€'

positive variables GENE(tec,h),CAPA(tec),STORAGE(str,h), S(str),STORED(str,h),CAPACITY(str),RSV(frr,h);
```

### Equations

```
equations        gene_vre        'variables renewable profiles generation'
                 gene_capa       'capacity and genration relation for technologies'
                 combustion      'the relationship of combustible technologies'
                 capa_frr        'capacity needed for the secondary reserve requirements'
                 storing         'the definition of stored energy in the storage options'
                 storage_const   'storage in the first hour is equal to the storage in the last hour'
                 lake_res        'constraint on water for lake reservoirs'
                 stored_cap      'maximum energy that is stored in storage units'
                 storage_capa1   'the capacity with hourly charging relationship of storage'
                 storage_capa2   'storage power limit'
                 biogas_const    'maximum energy can be produced by biogas'
                 reserves        'FRR requirement'
                 adequacy        'supply/demand relation'
                 obj             'the final objective function which is COST';

gene_vre(vre,h)..                GENE(vre,h)             =e=     CAPA(vre)*load_factor(vre,h);
gene_capa(tec,h)..               CAPA(tec)               =g=     GENE(tec,h);
combustion(h)..                  GENE('gas',h)           =e=     sum(comb,GENE(comb,h));
capa_frr(frr,h)..                CAPA(frr)               =g=     GENE(frr,h) + RSV(frr,h);
storing(h,h+1,str)..             STORED(str,h+1)         =e=     STORED(str,h) + STORAGE(str,h)*eta_in(str) - GENE(str,h)/eta_out(str);
storage_const(str,first,last)..  STORED(str,first)       =e=     STORED(str,last)+STORAGE(str,last)*eta_in(str)-GENE(str,last)/eta_out(str);
lake_res(m)..                    lake_inflows(m)         =g=     sum(h$(month(h) = ord(m)),GENE('lake',h))/1000;
stored_cap(str,h)..              STORED(str,h)           =l=     CAPACITY(str);
storage_capa1(str,h)..           S(str)                  =g=     STORAGE(str,h);
storage_capa2(str)..             S(str)                  =l=     CAPA(str);
biogas_const..                   sum(h,GENE('biogas',h)) =l=     max_biogas*1000;
reserves(h)..                    sum(frr, RSV(frr,h))    =e=     sum(vre,epsilon(vre)*CAPA(vre))+ demand(h)*load_uncertainty*(1+delta);
adequacy(h)..                    sum(ncomb,GENE(ncomb,h))+GENE('gas',h)    =g=     demand(h)+sum(str,STORAGE(str,h)) ;
obj..                            COST                    =e=     (sum(tec,(CAPA(tec)-capa_ex(tec))*capex(tec))+ sum(str,CAPACITY(str)*capex_en(str))+sum(tec,(CAPA(tec)*fOM(tec)))+ sum(str,S(str)*(s_capex(str)+s_opex(str))) + sum((tec,h),GENE(tec,h)*vOM(tec)))/1000;
```